# Steckbrief einer elementaren Funktion

Zuerst wird in der gegebenen Funktion für alle 'c' eine zufällige Konstante aus der Menge {1,2,3} eingesetzt. Daher wird diese zunächst als String erwartet.

In [117]:
from sympy import *
import re
import random
x,c = symbols('x c')

f_str = 'c+c*(x**2-c)/x**4'

### Zufällig ausgewählte Funktion:

In [118]:
f = sympify(re.sub(r"c(?!o)", lambda m: str(random.choice([1,2,3])), f_str))
print(f)
f

3 + (3*x**2 - 3)/x**4


3 + (3*x**2 - 3)/x**4

### Definitionsbereich

In [119]:
domain = calculus.util.continuous_domain(f, x, S.Reals)
print(str(domain))
domain

Union(Interval.open(-oo, 0), Interval.open(0, oo))


Union(Interval.open(-oo, 0), Interval.open(0, oo))

### Wertebereich

**Achtung!** Der Wertebereich kann unter Umständen nicht berechenbar sein 

In [120]:
f_range = None
try:
    f_range = calculus.util.function_range(f, x, S.Reals)
    print(str(f_range))
except NotImplementedError:
    print('Wertebereich nicht berechenbar')
f_range

Interval(-oo, 15/4)


Interval(-oo, 15/4)

### Unstetigkeitsstellen

In [121]:
try:
    f_singularities = singularities(f, x, S.Reals)
    if f_singularities.is_FiniteSet:
        print(len(f_singularities), ' Unstetigkeitsstelle(n)')
        for s in f_singularities:
            left_limit = limit(f, x, s, '-')
            right_limit = limit(f, x, s, '+')
            f_x = f.subs(x, s)
            if left_limit.is_real and left_limit == right_limit:
                print('Hebbare Lücke: ', s)
            elif left_limit.is_real and right_limit.is_real and left_limit != right_limit:
                print('Sprungstelle: ', s)
            elif any([not l for l in [left_limit, right_limit]]):
                print('Wesentliche Singularität: ', s)
            else:
                print('Polstelle: ', s)
            if f_x == left_limit:
                print('(linksseitig)')
            elif f_x == right_limit:
                print('(rechtsseitig)')
    else:
        print('Unendlich Unstetigkeitsstellen')
        print(f_singularities)
        f_singularities = None
except NotImplementedError:
    print('Unstetigkeitsstellen nicht berechenbar')

1  Unstetigkeitsstelle(n)
Polstelle:  0


### Grenzwerte

In [122]:
points = FiniteSet(-oo, oo)
if f_singularities:
    points += f_singularities
for p in points:
    l = limit(f, x, p)
    print('Grenzwert bei ', p, ': ', l)

Grenzwert bei  -oo :  3
Grenzwert bei  0 :  -oo
Grenzwert bei  oo :  3


### Asymptoten

In [123]:
# Calculation method taken from https://en.wikipedia.org/wiki/Asymptote#Elementary_methods_for_identifying_asymptotes
# Also mentioned in https://encyclopediaofmath.org/index.php?title=Asymptote
def asymptotes():
    a = set()
    for lim in [oo, -oo]:
        m = limit(f/x, x, lim)
        if m.is_real:
            n = limit(f-m*x, x, lim)
            if n.is_real:
                a.add(m*x+n)
    return a

a = asymptotes()
print('Asymptoten' if a else 'Keine Asymptoten')
a

Asymptoten


{3}

### Periodizität

In [124]:
p = periodicity(f, x)
print('Periodisch' if p else 'Nicht periodisch')
p

Nicht periodisch


### y-Achsenschnitt

In [125]:
y_intercept = f.subs(x, 0)
y_intercept

zoo

### Nullstellen

Eine Funktion kann eine endliche oder unendliche Menge an Nullstellen haben, daher muss hier differenziert werden.

Die Funktion `solveset()` liefert die genaue Menge der Nullstellen algebraisch definiert. Diese ist jedoch für spätere Filterung schlechter geeignet.

In [126]:
all_zeros = solveset(f, x, S.Reals)
infinite_zeros = not all_zeros.is_FiniteSet

if infinite_zeros:
    print('Menge der Nullstellen ist unendlich')
else:
    print('Menge der Nullstellen ist endlich')
all_zeros

Menge der Nullstellen ist endlich


{-sqrt(-1/2 + sqrt(5)/2), sqrt(-1/2 + sqrt(5)/2)}

Mit Hilfe von `solve()` wird immer eine endliche Menge zurückgegeben, die allerdings im Falle unendlicher Nullstellen nicht vollständig ist (z.B. bei $sin(x)$):

In [127]:
finite_zeros = solve(f, x) if infinite_zeros else all_zeros
finite_zeros

{-sqrt(-1/2 + sqrt(5)/2), sqrt(-1/2 + sqrt(5)/2)}

### Ableitung

In [128]:
fd = diff(f, x)
fd

6/x**3 - 4*(3*x**2 - 3)/x**5

### Extremstellen

In [129]:
fd_zeros = solveset(fd, x, S.Reals)
fdd = diff(fd, x)
extrema = FiniteSet()
if fd_zeros.is_empty:
    print('Keine Extremstellen')
elif fd_zeros.is_FiniteSet:
    for z in fd_zeros:
        val = fdd.subs(x, z)
        if val.is_negative:
            extrema += FiniteSet(z)
            print('Maximum: ', z)
        elif val.is_positive:
            extrema += FiniteSet(z)
            print('Minimum: ', z)
else:
    extrema = None
    print('Extremstellen nicht elementar berechenbar oder unendlich')
    print(fd_zeros)

Maximum:  sqrt(2)
Maximum:  -sqrt(2)


### Wendestellen

In [130]:
fdd_zeros = solveset(fdd, x, S.Reals)
inflections = FiniteSet()
if fdd_zeros.is_empty:
    print('Keine Wendepunkte')
elif fdd_zeros.is_FiniteSet:
    fddd = diff(fd, x)
    for z in fdd_zeros:
        val = fddd.subs(x, z)
        if not val.is_zero:
            print('Wendepunkt: z')
            inflections += FiniteSet(z)
else:
    inflections = None
    print('Wendepunkte nicht elementar berechenbar oder unendlich')
    print(fdd_zeros)

### Monotonieintervalle

In [134]:
# https://www.massmatics.de/merkzettel/#!144:Monotonie_von_Funktionen
def monotonicity():
    if f_singularities == None or extrema == None:
        print('Monotonieintervalle nicht berechenbar oder unendlich')
        return None
    interval_points = list(f_singularities.union(extrema).union(FiniteSet(-oo, oo)))
    interval_points.sort()
    intervals = [Interval(prev, curr) for prev, curr in zip(interval_points, interval_points[1:])]
    for interval in intervals:
        if is_strictly_increasing(f, interval, x):
            print('streng monoton steigend in ', interval)
        elif is_strictly_decreasing(f, interval, x):
            print('streng monoton fallend in ', interval)
        elif is_increasing(f, interval, x):
            print('monoton steigend in ', interval)
        elif is_decreasing(f, interval, x):
            print('monoton fallend in ', interval)

monotonicity()

monoton steigend in  Interval(-oo, -sqrt(2))
monoton fallend in  Interval(sqrt(2), oo)


### Konvexitäts-/Konkavitätsbereiche

In [132]:
# https://www.massmatics.de/merkzettel/#!193:Konvexitaet
def convexity():
    if f_singularities == None or inflections == None:
        print('Konvexitäts-/Konkavitätsbereiche nicht berechenbar oder unendlich')
        return None
    interval_points = list(f_singularities.union(inflections).union(FiniteSet(-oo, oo)))
    interval_points.sort()
    intervals = [Interval(prev, curr) for prev, curr in zip(interval_points, interval_points[1:])]
    for interval in intervals:
        if is_convex(f, x, domain=interval):
            print('konvex in ', interval)
        elif is_convex(-f, x, domain=interval):
            print('konkav in ', interval)

convexity()